In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

!unzip udea*.zip > /dev/null
!wc *.csv

udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


**Cargar archivos como Dataframes**

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
auxiliar_df = train_df.copy()

In [ ]:
train_df.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'coef_1', 'coef_2', 'coef_3', 'coef_4'],
      dtype='object')

## **Funciones de Limpieza**

### **Rendimiento Global**

In [ ]:
def rendimientoglobal(df):
  map_dict = {
    "alto":3,
    "medio-alto":2,
    "medio-bajo":1,
    "bajo": 0
  }
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_dict)

### **Rendimiento Global Reverso**

In [ ]:
def rendimientoglobalreverse(df):
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].round().astype(int)
    map_dict = {
        3: "alto",
        2: "medio-alto",
        1: "medio-bajo",
        0: "bajo"
    }
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_dict)
    return df

### **Estrato**

In [ ]:
def estrato(df):
    map_dict = {
        'Estrato 1': 1,
        'Estrato 2': 3,
        'Estrato 3': 4,
        'Estrato 4': 5,
        'Estrato 5': 6,
        'Estrato 6': 7,
        'Sin Estrato': 2
    }

    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].map(map_dict)

    # Calcula la moda de manera segura
    modas = df['FAMI_ESTRATOVIVIENDA'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0  # valor por defecto si toda la columna es nula

    # Reemplaza valores nulos por la moda
    def generar_valores(row):
        if pd.isna(row):
            return moda
        return row

    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].apply(generar_valores)


### **Internet**

In [ ]:
#NUEVO - la probabilidad no es una constante, es el promedio de personas que dijieron que si
import random
def internet(df):
  map_dict = {
    'Si': 1,
    'No': 0,
  }
  df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].map(map_dict)
  percent=np.round(df['FAMI_TIENEINTERNET'].mean(),4)

  def assign_value(row):
    est_value=row['FAMI_ESTRATOVIVIENDA']
    if pd.isnull(row['FAMI_TIENEINTERNET']):
        if est_value > 2:
            return 1
        elif np.round(random.random(),4) <= percent:
            return 1
        else:
            return 0
    else:
        return row['FAMI_TIENEINTERNET']

  df['FAMI_TIENEINTERNET'] = df.apply(assign_value, axis=1)

### **Matricula Propia**

In [ ]:
def matriculapropia(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    res_mean = auxiliar_df.groupby('ESTU_PAGOMATRICULAPROPIO')["RENDIMIENTO_GLOBAL"].mean()
    res_std = auxiliar_df.groupby('ESTU_PAGOMATRICULAPROPIO')["RENDIMIENTO_GLOBAL"].std()

    # Mapear los resultados en el DataFrame
    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].map(res_mean)

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores_normales(row):
        if pd.isna(row):  # Si el valor es nulo
            mean = res_mean.mean()
            std = res_std.mean()
            return np.random.normal(mean, std)
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].apply(generar_valores_normales)


### **Horas que Trabaja**

In [ ]:
def horassemanatrabaja(df):
    map_dict = {
        '0': 5,
        'Entre 11 y 20 horas': 3,
        'Entre 21 y 30 horas': 2,
        'Menos de 10 horas': 4,
        'Más de 30 horas': 1
    }

    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map(map_dict)

    # calcula la moda de forma segura
    modas = df['ESTU_HORASSEMANATRABAJA'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0  # o el valor por defecto que prefieras

    # rellena los valores nulos con la moda
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna(moda)

### **Valor de la Matricula**

In [ ]:
def valormatriculauniversidad(df):
    map_dict = {
        'Entre 1 millón y menos de 2.5 millones': 1,
        'Entre 2.5 millones y menos de 4 millones': 2,
        'Entre 4 millones y menos de 5.5 millones': 6,
        'Entre 5.5 millones y menos de 7 millones': 7,
        'Entre 500 mil y menos de 1 millón': 3,
        'Menos de 500 mil': 5,
        'Más de 7 millones': 8,
        'No pagó matrícula': 4
    }

    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(map_dict)

    # calcula la moda de forma segura
    modas = df['ESTU_VALORMATRICULAUNIVERSIDAD'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0  # valor por defecto si no hay datos válidos

    # reemplaza nulos con la moda
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(moda)

### **Educación de Padre**

In [ ]:
def educacionpadre(df):
    map_dict = {
        'Educación profesional completa': 11,
        'Educación profesional incompleta': 10,
        'Ninguno': 1,
        'No Aplica': 7,
        'No sabe': 9,
        'Postgrado': 12,
        'Primaria completa': 3,
        'Primaria incompleta': 2,
        'Secundaria (Bachillerato) completa': 6,
        'Secundaria (Bachillerato) incompleta': 4,
        'Técnica o tecnológica completa': 8,
        'Técnica o tecnológica incompleta': 5
    }

    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(map_dict)

    modas = df['FAMI_EDUCACIONPADRE'].mode()
    moda = modas.iloc[0] if not modas.empty else 0

    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna(moda)


### **Educación de Madre**

In [ ]:
def educacionmadre(df):
    map_dict = {
        'Educación profesional completa': 11,
        'Educación profesional incompleta': 10,
        'Ninguno': 1,
        'No Aplica': 4,
        'No sabe': 6,
        'Postgrado': 12,
        'Primaria completa': 3,
        'Primaria incompleta': 2,
        'Secundaria (Bachillerato) completa': 7,
        'Secundaria (Bachillerato) incompleta': 5,
        'Técnica o tecnológica completa': 9,
        'Técnica o tecnológica incompleta': 8
    }

    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(map_dict)

    # calcula la moda de forma segura
    modas = df['FAMI_EDUCACIONMADRE'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0  # o el valor que prefieras por defecto

    # rellena nulos con la moda
    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna(moda)

### **Periodo**

In [ ]:
def periodo(df):
    df['PERIODO'] = df['PERIODO'].astype(str)

    map_dict = {
        "20183": 5,
        "20184": 1,
        "20194": 9,
        "20195": 3,
        "20196": 4,
        "20202": 8,
        "20203": 6,
        "20212": 2,
        "20213": 7
    }

    df['PERIODO'] = df['PERIODO'].map(map_dict)

    modas = df['PERIODO'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0  # o el valor que prefieras como predeterminado

    df['PERIODO'] = df['PERIODO'].fillna(moda)

### **Departamento**

In [ ]:
def departamento(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    res_mean = auxiliar_df.groupby('ESTU_PRGM_DEPARTAMENTO')["RENDIMIENTO_GLOBAL"].mean()

    # Mapear los resultados en el DataFrame
    df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].map(res_mean)

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores_normales(row):
        if pd.isna(row):  # Si el valor es nulo
            mean = auxiliar_df['RENDIMIENTO_GLOBAL'].mean()
            return mean
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].apply(generar_valores_normales)


### **Programa Acádemico**

In [ ]:
def programaacademico(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    res_mean = auxiliar_df.groupby('ESTU_PRGM_ACADEMICO')["RENDIMIENTO_GLOBAL"].mean()

    # Mapear los resultados en el DataFrame
    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].map(res_mean)

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores_normales(row):
        if pd.isna(row):  # Si el valor es nulo
            mean = auxiliar_df['RENDIMIENTO_GLOBAL'].mean()
            return mean
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(generar_valores_normales)


### **Columnas binarias**

In [ ]:
def columnas_binarias(df):
    """Convierte las columnas binarias 'Si'/'No' a 1/0"""
    binarias = [
        'FAMI_TIENELAVADORA',
        'FAMI_TIENEAUTOMOVIL',
        'ESTU_PRIVADO_LIBERTAD',
        'FAMI_TIENECOMPUTADOR',
        'FAMI_TIENEINTERNET.1'
    ]
    for col in binarias:
        if col in df.columns:
            df[col] = df[col].map({'Si': 1, 'No': 0})
            df[col] = df[col].fillna(0)  # opcional: llena nulos con 0

## **Limpieza Total**

In [ ]:
def limpiar_df(df, predict=False, test=False):
  if not predict:
    if not test:
      rendimientoglobal(df)
    estrato(df)
    internet(df)
    matriculapropia(df)
    valormatriculauniversidad(df)
    educacionpadre(df)
    educacionmadre(df)
    horassemanatrabaja(df)
    departamento(df)
    programaacademico(df)
    periodo(df)
    columnas_binarias(df)
    return df
  elif predict:
    rendimientoglobal(df)

In [ ]:
limpiar_df(auxiliar_df, predict=True)
train_df2 = limpiar_df(train_df)
test_df2 = limpiar_df(test_df, test=True)

# **Adaboost Classifier**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

df = train_df.copy()

# Separar características (X) y la variable objetivo (y)
X = df.drop(columns=["RENDIMIENTO_GLOBAL","ID"], axis=1)
y = df["RENDIMIENTO_GLOBAL"]
# Supongamos que ya tienes tus datos preparados: X y y
# X es tu conjunto de características y y son las etiquetas

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Rellenar nulos con 0
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Inicializar el modelo de AdaBoost
# Puedes usar un árbol de decisión como modelo base
base_estimator = DecisionTreeClassifier(max_depth=5)  # Un árbol poco profundo
ada_classifier = AdaBoostClassifier(estimator=base_estimator,
                                     n_estimators=50,  # Número de modelos débiles
                                     learning_rate=0.3,  # Tasa de aprendizaje
                                     random_state=73)

start = time.time()


# Entrenar el modelo
ada_classifier.fit(X_train, y_train)

# Hacer predicciones
y_pred = ada_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy}')



end = time.time()
print(f"Entrenamiento completado en {end - start:.2f} segundos.")

Precisión: 0.4180361010830325
Entrenamiento completado en 157.20 segundos.


In [ ]:
# Crear copia limpia del test
df_test_est = test_df2.copy()

# Eliminar columnas que no estaban en el entrenamiento
df_test_est = df_test_est.drop(columns=["ID"], errors='ignore')  # Ignora si no existe
df_test_est = df_test_est.drop(columns=["Unnamed: 0"], errors='ignore')

# Realizar predicciones
predicciones = ada_classifier.predict(df_test_est)

In [ ]:
# Crear DataFrame con ID y predicciones
id_column = "ID" if "ID" in test_df.columns else test_df.index
y = pd.DataFrame({ "ID": test_df[id_column], "RENDIMIENTO_GLOBAL": predicciones })

# Función para revertir el rendimiento global a etiquetas originales
def rendimientoglobalreverse(df):
    df["RENDIMIENTO_GLOBAL"] = df["RENDIMIENTO_GLOBAL"].round().astype(int)
    map_dict = {3: "alto", 2: "medio-alto", 1: "medio-bajo", 0: "bajo"}
    df["RENDIMIENTO_GLOBAL"] = df["RENDIMIENTO_GLOBAL"].map(map_dict)
    return df

# Aplicar reversión y guardar CSV
y = rendimientoglobalreverse(y)
y.to_csv("submission.csv", index=False)
print("✅ Archivo 'submission.csv' guardado correctamente.")


✅ Archivo 'submission.csv' guardado correctamente.


# **Predicción Kaggle**

In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia -f submission.csv -m "Intento con AdaBoost de Camilo"


100% 4.10M/4.10M [00:00<00:00, 9.74MB/s]
Successfully submitted to UDEA/ai4eng 20251 - Pruebas Saber Pro Colombia